In [1]:
# Load model directly
import torch
from transformers import AutoModel, AutoTokenizer
model_id="/mnt/petrelfs/songmingyang/songmingyang/model/mm/ShareCaptioner"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True)
model.tokenizer = tokenizer
model=model.to("cuda").to(torch.float16)

/mnt/petrelfs/songmingyang/anaconda3/envs/smoe/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
sentencepiece_processor.cc(922) LOG(ERROR) src/sentencepiece_processor.cc(289) [model_] Model is not initialized.
Returns default value 0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Set max length to 20
Init VIT ... Position interpolate from 16x16 to 32x32
Done
Init Perceive Sampler ... Done
Init InternLM ... Done


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.36s/it]


In [13]:
from transformers import AutoProcessor 
from PIL import Image 
train_png="/mnt/petrelfs/songmingyang/code/mm/robustLMM/robustlmm/data_adjustment/diffusion_augment/train.jpg"
img = Image.open(train_png).convert('RGB')


In [16]:
batch_size=4
img_embed=model.vis_processor(img).unsqueeze(0).to(model.dtype).to(model.device)
img_embed=img_embed.repeat(batch_size,1,1,1)


In [17]:
img_embed.shape

torch.Size([4, 3, 448, 448])

In [18]:
seg1 = '<|User|>:'
seg2 = f'Analyze the image in a comprehensive and detailed manner.{model.eoh}\n<|Bot|>:'
seg_emb1 = model.encode_text(seg1, add_special_tokens=True)
seg_emb2 = model.encode_text(seg2, add_special_tokens=False)
tmp_seg_emb1 = seg_emb1.repeat(batch_size, 1, 1).to(model.device).to(model.dtype)
tmp_seg_emb2 = seg_emb2.repeat(batch_size, 1, 1).to(model.device).to(model.dtype)

In [19]:
img_embed = model.encode_img(img_embed)
input_emb = torch.cat([tmp_seg_emb1, img_embed, tmp_seg_emb2], dim=1)
out_embeds = model.internlm_model.generate(inputs_embeds=input_emb,
                                           max_length=800,
                                           num_beams=3,
                                           min_length=1,
                                           do_sample=True,
                                           repetition_penalty=1.5,
                                           length_penalty=1.0,
                                           temperature=1.,
                                           eos_token_id=model.tokenizer.eos_token_id,
                                           num_return_sequences=1,
                                           )

In [37]:
out_embeds[out_embeds == -1] = 2

In [65]:
def get_rid_of_substr(substr,list_of_target_str):
    renewed_target = []
    for temp_str in list_of_target_str:
        for temp_sub in substr:
            temp_str = temp_str.replace(temp_sub,"")
        renewed_target.append(temp_str)
    return renewed_target

In [66]:
out_text = tokenizer.batch_decode(out_embeds, skip_special_tokens=True)
get_rid_strs = ["<s>","</s>","<TOKENS_UNUSED_1>","<unk>"]
out_text = get_rid_of_substr(get_rid_strs,out_text)
# out_text = model.decode_text(out_embeds)

In [71]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

In [67]:
out_text

["  The image captures a bustling scene at a train station. Dominating the frame are three trains, each with its own distinct color scheme and purpose. \n\nOn the left, a green and yellow passenger train is in motion, its multiple cars filled with unseen passengers. The middle of the image is dominated by a red and white freight train, a testament to the industrial nature of the scene. It's stationary, perhaps waiting for its cargo to be loaded or unloaded.\n\nOn the right, a green and yellow passenger train mirrors its counterpart on the left, suggesting a symmetrical balance in the composition of the image. \n\nThe trains are situated on multiple tracks that crisscross and diverge, creating a complex network of steel and iron. Overhead wires stretch across the scene, hinting at the electric power that propels these mechanical beasts.\n\nIn the background, a bridge and trees can be seen, adding a touch of nature to this industrial landscape. The bridge, possibly a vital link for the t

In [9]:
prompt="Analyze the image in a comprehensive and detailed manner."
output_embeds = model.generate(text=prompt,image=train_png)

In [10]:
output_embeds

' <s> The image captures a bustling scene at a train station. Dominating the frame are three trains, each with its own distinct color scheme. The first train, painted in vibrant hues of red and green, is moving towards the right side of the image. The second train, adorned in shades of green and yellow, is heading in the opposite direction, towards the left side of the image. The third train, mirroring the first with its red and green color scheme, is stationary, adding a sense of balance to the scene.\n\nThe trains are situated on multiple tracks that crisscross each other, creating a complex network of steel and iron. Overhead wires stretch across the image, hinting at the electric power that propels these mechanical beasts.\n\nIn the background, a bridge and trees can be seen, providing a stark contrast to the industrial nature of the train station. The bridge, with its sturdy construction, stands as a testament to human ingenuity, while the trees add a touch of nature to the otherw

In [14]:
seg2 = f'Analyze the image in a comprehensive and detailed manner.{model.eoh}\n<|Bot|>:'
seg_emb2 = model.encode_text(seg2, add_special_tokens=False)

torch.Size([1, 18, 4096])

torch.Size([1, 258, 4096])